In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
# colab pro 더 빠른 GPU 사용
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 24 10:09:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [131]:
! pip install transformers
! pip install torchmetrics
! pip install torchtext

In [132]:
! pip install Datasets

In [144]:
! pip install sentencepiece

In [145]:
# BirBird requirements 설치
! git clone https://github.com/google-research/bigbird.git
! cd bigbird
! pip3 install -e .
! mkdir -p bigbird/ckpt
# ! gsutil cp -r gs://bigbird-transformer/ bigbird/ckpt/

fatal: destination path 'bigbird' already exists and is not an empty directory.
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


In [146]:
# colab pro 추가 메모리
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [147]:
import pandas as pd
import numpy as np
from transformers import (
    BigBirdModel,
    BigBirdTokenizer,
    BigBirdForPreTraining,
    TrainingArguments,
    Trainer
)

from sklearn.model_selection import train_test_split

from datasets import Dataset
from datasets import load_metric

In [148]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/Data/Training_Data_Clean.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/Data/Test_Data_Clean.csv'

train_data = pd.read_csv(train_path, encoding="latin-1") # 데이터를 더 잘 읽어올 수 있도록 인코딩 지정
test_df = pd.read_csv(test_path, encoding="latin-1")

In [149]:
train_data = train_data.rename(columns={'Full_Text': 'text', "Summary_Text": "summary" })
train_data = train_data.drop('Unnamed: 0', axis=1)
train_data.head()

,Original_Filename_x,Gold_Filename,Document,text,summary
0,15786_819157_2,15786_819156_1,15786,6 chief executive officerâs review the bero...,3 chairmanâs statement in the 18 months sin...
1,15783_819108_2,15783_819107_1,15783,chief executive officerâs statement the ber...,chairman s statement it has always been your ...
2,15782_819091_2,15782_819090_1,15782,chief executive officerâs statement the com...,chairmanâs statement the celestial ipilan o...
3,15787_819176_2,15787_819175_1,15787,page 6 toledo mining corporation plc annual r...,page 5 i should like to begin my message with...
4,15816_820156_2,15816_820155_1,15816,3 chief executiveâs statement tomkins plc r...,2 tomkins plc report accounts 2003 chairmanâ...


In [150]:
train_df, val_df = train_test_split(train_data, test_size=0.1)
train_df.shape, val_df.shape

((2167, 5), (241, 5))

In [151]:
test_df = test_df[["Full_Text", "Summary_Text"]]
test_df = test_df.rename(columns={'Full_Text': 'text', "Summary_Text": "summary" })
test_df.head()

,text,summary
0,25695 19 march 2018 3 29 pm proof 7 02 s . c ...,25695 19 march 2018 3 29 pm proof 7 a . c . q...
1,10 staffline group plc annual report 2017 chi...,6 staffline group plc annual report 2017 a ye...
2,results corero revenue for the year ended 31 ...,corero network security plc annual report acc...
3,strategic report chief executiveâs statemen...,summary fy17 was a landmark year in alphaâs...
4,2017 has been a year of significant change fo...,sir richard lapthorne chairman 2017 has been ...


In [152]:
print(train_df.shape), print(val_df.shape), print(test_df.shape)

(2167, 5)
(241, 5)
(280, 2)


(None, None, None)

In [154]:
# 모델
model = BigBirdForPreTraining.from_pretrained('google/bigbird-roberta-base')

In [155]:
test_sen = train_df['text'][0]
test_sen

' 6 chief executive officerâ\x80\x99s review the berong nickel mine , in which toledo has a 56.1 percent economic interest , successfully recommenced operations in may 2011.this became possible due to favourable market conditions for high grade saprolite nickel ores and strong nickel prices which have stayed above usdollar 20,000 tonne since august 2010. prior to reopening the berong mine , toledo developed a new strategy with its jv partners in berong nickel corporation bnc to target production of nickel ore containing 1.8 percent nickel , which is currently being implemented . the 1.8 percent plan was developed consistent with growing market demands for this ore type . the recent significant changes in market demand for high grade saprolite nickel ore have opened up sound prospects for our deposits . historically , the development of exports of high grade saprolite nickel ore began from new caledonia during the 1950 s by japanese ferronickel producers . later , they extended their or

In [156]:
# 토크나이저

# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 


# Define the model repo
model_name = "google/bigbird-roberta-base" 


# Download pytorch model
tokenizing_model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [157]:
# Transform input tokens 
inputs = tokenizer(test_sen, return_tensors="pt")

# Model apply
outputs = model(**inputs)

/usr/local/lib/python3.7/dist-packages/transformers/models/big_bird/modeling_big_bird.py:978: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  * num_indices_to_pick_from


bigbird 기법는 tokenizer 구현이 안돼있었던 것 같다. 따라서 AutoTokenizer를 이용해 pretrain된 tokenizer와 모델을 불러왔다. setup.py 또한 코랩에서 설치가 안되는 것 같은데 해결책을 찾아봐야겠다.